In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizerFast, BertForSequenceClassification, AdamW, get_scheduler
from tqdm.auto import tqdm
import wandb


import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer, XLNetModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np
from transformers import AutoTokenizer, AutoModel

/home/jiyoon/miniconda3/envs/RecLLM/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-04 13:09:25.850733: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-04 13:09:25.870006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741093765.892143  909553 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741093765.898733  909553 cuda_blas.cc:

In [2]:
import pandas as pd
df = pd.read_csv('cleaned_imdb_genre.csv')
df

,Unnamed: 0,id,title,desc,genre
0,0,tt0000005,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,Short
1,1,tt0000004,Un bon bock,Lost 1892 French short animated film directed ...,"Animation,Short"
2,2,tt0000002,Le clown et ses chiens,Lost short film consisting of 300 painted imag...,"Animation,Short"
3,3,tt0000003,Poor Pierrot,"One night, Arlequin come to see his lover Colo...","Animation,Comedy,Romance"
4,4,tt0000001,Carmencita,Performing on what looks like a small wooden s...,"Documentary,Short"
...,...,...,...,...,...
207356,390752,tt0407808,Frog and Toad Are Friends,Claymation version of Arnold Lobel's story of ...,"Animation,Comedy,Family"
207357,390753,tt0407810,From Ardoyne to the Áras: Inside the McAleese ...,Documentary on the private and public life of ...,Documentary
207358,390754,tt0407811,Frontstadt,A young filmmaker tries to gain a very persona...,Drama
207359,390755,tt0407815,Possible Changes,"Two friends, Moon-ho and Jong-kyu, in their mi...",Drama


In [3]:
# primaryTitle과 description을 하나의 텍스트로 합치기
df['text'] = df['title'].astype(str) + " " + df['desc'].astype(str)

# genre 컬럼 전처리: 쉼표로 구분된 문자열을 리스트로 변환
def process_genres(genres_str):
    if pd.isna(genres_str):
        return []
    return [g.strip() for g in genres_str.split(',') if g.strip() != ""]

df['genre_list'] = df['genre'].apply(process_genres)

In [4]:
all_genres = set()
for genres in df['genre_list']:
    for genre in genres:
        all_genres.add(genre)
all_genres = sorted(list(all_genres))
genre2id = {genre: idx for idx, genre in enumerate(all_genres)}
num_labels = len(all_genres)
print("전체 장르:", all_genres)

전체 장르: ['Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show', 'Thriller', 'War', 'Western']


In [5]:
# 각 샘플에 대해 멀티핫 인코딩된 레이블 생성
def encode_labels(genres):
    label = [0] * num_labels
    for g in genres:
        if g in genre2id:
            label[genre2id[g]] = 1
    return label

df['labels'] = df['genre_list'].apply(encode_labels)

# 모델 학습에 필요한 열만 선택
df_model = df[['text', 'genre_list', 'labels']]

In [6]:
df_model

,text,genre_list,labels
0,Blacksmith Scene Three men hammer on an anvil ...,[Short],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Un bon bock Lost 1892 French short animated fi...,"[Animation, Short]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Le clown et ses chiens Lost short film consist...,"[Animation, Short]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"Poor Pierrot One night, Arlequin come to see h...","[Animation, Comedy, Romance]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Carmencita Performing on what looks like a sma...,"[Documentary, Short]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...
207356,Frog and Toad Are Friends Claymation version o...,"[Animation, Comedy, Family]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
207357,From Ardoyne to the Áras: Inside the McAleese ...,[Documentary],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
207358,Frontstadt A young filmmaker tries to gain a v...,[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
207359,"Possible Changes Two friends, Moon-ho and Jong...",[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [7]:
def split_by_genre(df):
    rows = []
    for idx, row in df.iterrows():
        text = row['text']
        genres = row['genre_list']
        labels = row['labels']
        
        for genre in genres:
            
            # 새 데이터로 생성 (text, genre, genre_label)
            rows.append({
                'text': text,
                'genre': genre,
                'labels': labels
            })
    
    return pd.DataFrame(rows)

# 장르별로 텍스트와 라벨을 분리
# genre_df = split_by_genre(df)

# genre_df

In [7]:
label_descriptions = {
    "Action": "This movie has thrilling action sequences with intense fight scenes.",
    "Adult": "This film is intended for mature audiences, featuring explicit themes.",
    "Adventure": "This movie takes the audience on an exciting journey full of discovery.",
    "Animation": "This film is beautifully animated with vibrant characters and stunning visuals.",
    "Biography": "This movie tells the true story of a remarkable person's life.",
    "Comedy": "This movie is full of humor and laughter, guaranteed to entertain.",
    "Crime": "This film revolves around criminal activities, investigations, and justice.",
    "Documentary": "This is a factual film that explores real-life events and issues.",
    "Drama": "This movie tells an emotional and heartfelt story with deep character development.",
    "Family": "This movie is suitable for all ages, bringing warmth and joy to families.",
    "Fantasy": "This film takes place in a magical world with fantastical elements and creatures.",
    "Film-Noir": "This movie features a dark and mysterious atmosphere with complex characters.",
    "Game-Show": "This show features competitive games and exciting challenges.",
    "History": "This film brings historical events and figures to life with great detail.",
    "Horror": "This movie contains scary and suspenseful moments that will keep you on edge.",
    "Music": "This film revolves around music, featuring incredible performances and soundtracks.",
    "Musical": "This movie is filled with songs and dance performances that tell a story.",
    "Mystery": "This film keeps the audience guessing with twists and hidden secrets.",
    "News": "This program covers current events and breaking news from around the world.",
    "Reality-TV": "This show follows real people and their lives, providing entertainment and drama.",
    "Romance": "A heartwarming romantic story unfolds in this film, full of love and emotions.",
    "Sci-Fi": "This movie explores futuristic worlds, advanced technology, and space travel.",
    "Short": "This is a short film that tells a compelling story in a brief runtime.",
    "Sport": "This film is centered around sports, athletes, and competitive events.",
    "Talk-Show": "This show features discussions, interviews, and engaging conversations.",
    "Thriller": "This movie is filled with suspense, unexpected twists, and tension.",
    "War": "This film portrays intense battles and the impact of war on people.",
    "Western": "This movie is set in the Old West, featuring cowboys, duels, and frontier life."
}

In [8]:
hard_negatives = {
    "Action": ["Adventure", "Thriller"],
    "Adult": ["Drama", "Romance"],
    "Adventure": ["Fantasy", "Action"],
    "Animation": ["Family", "Fantasy"],
    "Biography": ["History", "Drama"],
    "Comedy": ["Family", "Musical"],
    "Crime": ["Thriller", "Drama"],
    "Documentary": ["History", "News"],
    "Drama": ["Romance", "Biography"],
    "Family": ["Animation", "Comedy"],
    "Fantasy": ["Sci-Fi", "Adventure"],
    "Film-Noir": ["Mystery", "Thriller"],
    "Game-Show": ["Reality-TV", "Talk-Show"],
    "History": ["Biography", "Documentary"],
    "Horror": ["Thriller", "Mystery"],
    "Music": ["Musical", "Drama"],
    "Musical": ["Music", "Comedy"],
    "Mystery": ["Thriller", "Crime"],
    "News": ["Documentary", "Talk-Show"],
    "Reality-TV": ["Game-Show", "Talk-Show"],
    "Romance": ["Drama", "Comedy"],
    "Sci-Fi": ["Fantasy", "Action"],
    "Short": ["Documentary", "Animation"],
    "Sport": ["Drama", "Action"],
    "Talk-Show": ["Reality-TV", "News"],
    "Thriller": ["Horror", "Mystery"],
    "War": ["History", "Drama"],
    "Western": ["Adventure", "Action"]
}


In [9]:
model_df = df[['title', 'desc', 'genre_list']]
model_df

,title,desc,genre_list
0,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,[Short]
1,Un bon bock,Lost 1892 French short animated film directed ...,"[Animation, Short]"
2,Le clown et ses chiens,Lost short film consisting of 300 painted imag...,"[Animation, Short]"
3,Poor Pierrot,"One night, Arlequin come to see his lover Colo...","[Animation, Comedy, Romance]"
4,Carmencita,Performing on what looks like a small wooden s...,"[Documentary, Short]"
...,...,...,...
207356,Frog and Toad Are Friends,Claymation version of Arnold Lobel's story of ...,"[Animation, Comedy, Family]"
207357,From Ardoyne to the Áras: Inside the McAleese ...,Documentary on the private and public life of ...,[Documentary]
207358,Frontstadt,A young filmmaker tries to gain a very persona...,[Drama]
207359,Possible Changes,"Two friends, Moon-ho and Jong-kyu, in their mi...",[Drama]


In [ ]:
df

,Unnamed: 0,id,title,desc,genre,text,genre_list,labels
0,0,tt0000005,Blacksmith Scene,Three men hammer on an anvil and pass a bottle...,Short,Blacksmith Scene Three men hammer on an anvil ...,[Short],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,tt0000004,Un bon bock,Lost 1892 French short animated film directed ...,"Animation,Short",Un bon bock Lost 1892 French short animated fi...,"[Animation, Short]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,tt0000002,Le clown et ses chiens,Lost short film consisting of 300 painted imag...,"Animation,Short",Le clown et ses chiens Lost short film consist...,"[Animation, Short]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,tt0000003,Poor Pierrot,"One night, Arlequin come to see his lover Colo...","Animation,Comedy,Romance","Poor Pierrot One night, Arlequin come to see h...","[Animation, Comedy, Romance]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,tt0000001,Carmencita,Performing on what looks like a small wooden s...,"Documentary,Short",Carmencita Performing on what looks like a sma...,"[Documentary, Short]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
207356,390752,tt0407808,Frog and Toad Are Friends,Claymation version of Arnold Lobel's story of ...,"Animation,Comedy,Family",Frog and Toad Are Friends Claymation version o...,"[Animation, Comedy, Family]","[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
207357,390753,tt0407810,From Ardoyne to the Áras: Inside the McAleese ...,Documentary on the private and public life of ...,Documentary,From Ardoyne to the Áras: Inside the McAleese ...,[Documentary],"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
207358,390754,tt0407811,Frontstadt,A young filmmaker tries to gain a very persona...,Drama,Frontstadt A young filmmaker tries to gain a v...,[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
207359,390755,tt0407815,Possible Changes,"Two friends, Moon-ho and Jong-kyu, in their mi...",Drama,"Possible Changes Two friends, Moon-ho and Jong...",[Drama],"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
from transformers import AutoTokenizer, AutoModel
from unsloth import FastLanguageModel

# 모델 로드 (8-bit 적용)
MODEL_NAME = "unsloth/phi-4-unsloth-bnb-4bit"
load_in_4bit = True
max_seq_length = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = MODEL_NAME,
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.731 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.61s/it]


In [13]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", # self-attention 레이어
                      "gate_proj", "up_proj", "down_proj",], # MLP 레이어 (장르 정보 인코딩 강화)
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

model.to(device)
model.print_trainable_parameters()  # 학습 가능한 파라미터만 확인

Unsloth 2025.2.15 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


trainable params: 65,536,000 || all params: 14,725,043,200 || trainable%: 0.4451


In [28]:
for name, param in model.named_parameters():
    if param.requires_grad:
        param.data = param.data.to(model.dtype)  # 모델 dtype(BFloat16)과 맞추기


In [14]:
for name, param in model.named_parameters():
    if "lora" in name:
        print(name, param.shape)


base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight torch.Size([16, 5120])
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight torch.Size([5120, 16])
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight torch.Size([16, 5120])
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight torch.Size([1280, 16])
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight torch.Size([16, 5120])
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight torch.Size([1280, 16])
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight torch.Size([16, 5120])
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight torch.Size([5120, 16])
base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight torch.Size([16, 5120])
base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight torch.Size([17920, 16])
base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight

In [15]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total Trainable Parameters: {trainable_params}")


Total Trainable Parameters: 65536000


In [16]:
import random

def get_filtered_negative_samples(genre_list, all_genres, num_neg_samples=3):
    
    negative_candidates = list(set(all_genres) - set(genre_list))

    hard_neg_candidates = []
    for genre in genre_list:
        if genre in hard_negatives:
            hard_neg_candidates.extend(hard_negatives[genre])

    # Hard Negative 후보 중에서 실제 Negative 후보와 겹치는 것만 선택
    hard_neg_candidates = list(set(hard_neg_candidates) & set(negative_candidates))

    # 최종 Negative 샘플링 (Hard Negative + 추가 Negative)
    if len(hard_neg_candidates) < num_neg_samples:
        # Hard Negative가 부족하면 일반 Negative에서 추가
        additional_negatives = list(set(negative_candidates) - set(hard_neg_candidates))
        sampled_additional_negatives = random.sample(additional_negatives, num_neg_samples - len(hard_neg_candidates))
        final_neg_samples = hard_neg_candidates + sampled_additional_negatives
    else:
        # Hard Negative가 충분하면 거기서만 샘플링
        final_neg_samples = random.sample(hard_neg_candidates, num_neg_samples)

    # 장르 설명 텍스트 변환
    neg_texts = [label_descriptions[neg] for neg in final_neg_samples]

    return neg_texts


In [17]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split


# 데이터셋 클래스 정의
class ContrastiveDataset(Dataset):
    def __init__(self, df, tokenizer, all_genres=None, max_length=128, mode="train"):
        self.df = df
        self.tokenizer = tokenizer
        self.all_genres = all_genres
        self.max_length = max_length
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        title = row["title"]
        description = row["desc"]
        genre_list = row["genre_list"]

        if self.mode == "train":

            neg_texts = get_filtered_negative_samples(genre_list, all_genres)

            story_prompt = f"Movie Title: {title}, Story: {description}"
            label_prompt_pos = ["Label: " + label_descriptions[pos] for pos in genre_list]
            label_prompt_neg = ["Label: " + neg for neg in neg_texts]  # 여러 개의 Negative 샘플

            text_enc = self.tokenizer(story_prompt, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
            pos_enc = self.tokenizer(label_prompt_pos, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
            neg_enc = self.tokenizer(label_prompt_neg, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")

            return {
                "text_input_ids": text_enc["input_ids"].squeeze(0),
                "text_attention_mask": text_enc["attention_mask"].squeeze(0),
                "positive_input_ids": pos_enc["input_ids"],
                "positive_attention_mask": pos_enc["attention_mask"],
                "negative_input_ids": neg_enc["input_ids"],
                "negative_attention_mask": neg_enc["attention_mask"]
            }
        
        else:
            return {
                "title": title,
                "description": description,
                "answer": genre_list
            }



train_df, val_df = train_test_split(model_df, test_size=0.2, random_state=42)

train_dataset = ContrastiveDataset(train_df, tokenizer, all_genres=all_genres)
val_dataset = ContrastiveDataset(val_df, tokenizer, all_genres=all_genres)
test_dataset = ContrastiveDataset(val_df, tokenizer, mode="test")



In [18]:
from torch.nn.utils.rnn import pad_sequence

def train_collate_fn(batch):
    """배치 내 `positive_input_ids`의 크기를 맞추는 함수"""

    text_input_ids = torch.stack([b["text_input_ids"] for b in batch])
    text_attention_mask = torch.stack([b["text_attention_mask"] for b in batch])

    # 🔹 Positive 샘플 패딩 적용 (가장 큰 `num_positives` 기준)
    max_pos_samples = max([b["positive_input_ids"].shape[0] for b in batch])  # 배치 내 가장 긴 긍정 샘플 개수 찾기
    pos_input_ids = [torch.cat([b["positive_input_ids"], torch.zeros(max_pos_samples - b["positive_input_ids"].shape[0], b["positive_input_ids"].shape[1])]) if b["positive_input_ids"].shape[0] < max_pos_samples else b["positive_input_ids"] for b in batch]
    pos_attention_mask = [torch.cat([b["positive_attention_mask"], torch.zeros(max_pos_samples - b["positive_attention_mask"].shape[0], b["positive_attention_mask"].shape[1])]) if b["positive_attention_mask"].shape[0] < max_pos_samples else b["positive_attention_mask"] for b in batch]

    pos_input_ids = torch.stack(pos_input_ids)
    pos_attention_mask = torch.stack(pos_attention_mask)

    # 🔹 Negative 샘플 (3개로 고정)
    neg_input_ids = torch.stack([b["negative_input_ids"] for b in batch])
    neg_attention_mask = torch.stack([b["negative_attention_mask"] for b in batch])

    return {
        "text_input_ids": text_input_ids,
        "text_attention_mask": text_attention_mask,
        "positive_input_ids": pos_input_ids.to(torch.long),
        "positive_attention_mask": pos_attention_mask,
        "negative_input_ids": neg_input_ids.to(torch.long),
        "negative_attention_mask": neg_attention_mask
    }


def collate_fn(batch):
    """
    Test 데이터에서 배치 크기 불일치 문제 해결
    - `title`, `description`, `answer`를 리스트로 유지하여 DataLoader가 처리 가능하도록 함
    """
    titles = [item["title"] for item in batch]
    descriptions = [item["description"] for item in batch]
    answers = [item["answer"] for item in batch]  # 리스트 형태 유지

    return {
        "title": titles,
        "description": descriptions,
        "answer": answers
    }


train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=train_collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=train_collate_fn)  # 검증 데이터는 shuffle X
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

In [19]:
dtype = model.dtype
dtype

torch.bfloat16

In [20]:
# InfoNCE 손실 함수 정의
def info_nce_loss(query, positives, negatives, temperature=0.07):
    query = F.normalize(query, p=2, dim=-1)
    positives = F.normalize(positives, p=2, dim=-1)
    negatives = F.normalize(negatives, p=2, dim=-1)

    # 긍정 샘플들들과의 유사도
    pos_sim = torch.exp(torch.matmul(query.unsqueeze(1), positives.permute(0, 2, 1)).squeeze(1) / temperature)  # (batch_size, num_pos_samples)

    # 부정 샘플들과의 유사도 (여러 개의 부정 샘플 포함)
    neg_sim = torch.exp(torch.matmul(query.unsqueeze(1), negatives.permute(0, 2, 1)).squeeze(1) / temperature)  # (batch_size, num_neg_samples)

    # InfoNCE 손실 계산 (모든 부정 샘플을 고려)
    pos_sim_sum = torch.sum(pos_sim, dim=-1)  # (batch_size)
    neg_sim_sum = torch.sum(neg_sim, dim=-1)  # (batch_size)

    # 🚀 Loss 계산
    loss = -torch.log(pos_sim_sum / (pos_sim_sum + neg_sim_sum + 1e-8))

    return loss.mean()


In [21]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-5)

In [22]:
for param_group in optimizer.param_groups:
    param_group['params'] = [p.to(model.dtype) for p in param_group['params']]


In [24]:
for param_group in optimizer.param_groups:
    for param in param_group['params']:
        print(f"Optimizer Param dtype: {param.dtype}, Model dtype: {model.dtype}")

Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bfloat16
Optimizer Param dtype: torch.bfloat16, Model dtype: torch.bflo

In [29]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.dtype}, requires_grad={param.requires_grad}")


base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight: torch.bfloat16, requires_grad=True
base_model.model.model.layers.0.mlp.gate

In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

# 🚀 검증(Validation) 함수 정의
def validation_step(model, val_dataloader):
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(val_dataloader, avg_train_loss, desc="Validating", leave=False):

            query_emb = model(batch["text_input_ids"], batch["text_attention_mask"]).last_hidden_state[:, 0]

            batch_size, num_positives, seq_len = batch["positive_input_ids"].shape  
            pos_input_ids = batch["positive_input_ids"].reshape(batch_size * num_positives, seq_len)
            pos_attention_mask = batch["positive_attention_mask"].reshape(batch_size * num_positives, seq_len)

            pos_output = model(input_ids=pos_input_ids, attention_mask=pos_attention_mask, output_hidden_states=True)
            pos_emb = pos_output.hidden_states[-1][:, 0].reshape(batch_size, num_positives, -1)  # 원래 배치 형태로 복구

            batch_size, num_negatives, seq_len = batch["negative_input_ids"].shape  # 현재 Negative Samples 크기 확인
            #`negative_input_ids`를 `batch_size * num_negatives`로 Reshape
            neg_input_ids = batch["negative_input_ids"].reshape(batch_size * num_negatives, seq_len)
            neg_attention_mask = batch["negative_attention_mask"].reshape(batch_size * num_negatives, seq_len)

            # Negative Embedding 얻기
            neg_output = model(input_ids=neg_input_ids, attention_mask=neg_attention_mask, output_hidden_states=True)
            neg_emb = neg_output.hidden_states[-1][:, 0].reshape(batch_size, num_negatives, -1)  # 원래 배치 형태로 복구


            # InfoNCE 손실 계산
            loss = info_nce_loss(query_emb, pos_emb, neg_emb)
            total_val_loss += loss.item()

            # 🔥 예측값 생성 (query와 pos의 코사인 유사도를 기반으로 예측)
            # query_emb = F.normalize(query_emb, p=2, dim=-1)
            # pos_emb = F.normalize(pos_emb, p=2, dim=-1)
            # similarity = torch.matmul(query_emb, pos_emb.T).cpu().numpy()
            # preds = (similarity > 0.5).astype(int)  # Threshold = 0.5
            # labels = torch.ones_like(similarity)

            # all_preds.extend(preds.flatten())
            # all_labels.extend(labels.flatten())

    avg_val_loss = total_val_loss / len(val_dataloader)
    # accuracy = accuracy_score(all_labels, all_preds)
    # f1 = f1_score(all_labels, all_preds, average="macro")
    wandb.log({"Train Loss": avg_train_loss, "Val Loss": avg_val_loss})

    return avg_val_loss

# 학습 루프 (tqdm 적용 + Validation 추가)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    train_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in train_bar:
        
        optimizer.zero_grad()

        batch["text_input_ids"] = batch["text_input_ids"].to(device)
        batch["text_attention_mask"] = batch["text_attention_mask"].to(device)
        batch["positive_input_ids"] = batch["positive_input_ids"].to(device)
        batch["positive_attention_mask"] = batch["positive_attention_mask"].to(device)
        batch["negative_input_ids"] = batch["negative_input_ids"].to(device)
        batch["negative_attention_mask"] = batch["negative_attention_mask"].to(device)

        query_emb = model(batch["text_input_ids"], batch["text_attention_mask"], output_hidden_states=True).hidden_states[-1][:, 0]

        batch_size, num_positives, seq_len = batch["positive_input_ids"].shape  
        pos_input_ids = batch["positive_input_ids"].view(batch_size * num_positives, seq_len)
        pos_attention_mask = batch["positive_attention_mask"].view(batch_size * num_positives, seq_len)

        pos_output = model(input_ids=pos_input_ids, attention_mask=pos_attention_mask, output_hidden_states=True)
        pos_emb = pos_output.hidden_states[-1][:, 0].view(batch_size, num_positives, -1)  # 원래 배치 형태로 복구


        batch_size, num_negatives, seq_len = batch["negative_input_ids"].shape  # 현재 Negative Samples 크기 확인
        #`negative_input_ids`를 `batch_size * num_negatives`로 Reshape
        neg_input_ids = batch["negative_input_ids"].view(batch_size * num_negatives, seq_len)
        neg_attention_mask = batch["negative_attention_mask"].view(batch_size * num_negatives, seq_len)

        # Negative Embedding 얻기
        neg_output = model(input_ids=neg_input_ids, attention_mask=neg_attention_mask, output_hidden_states=True)
        neg_emb = neg_output.hidden_states[-1][:, 0].view(batch_size, num_negatives, -1)  # 원래 배치 형태로 복구

        # InfoNCE 손실 계산
        loss = info_nce_loss(query_emb, pos_emb, neg_emb)
        loss = loss.to(model.dtype)

        # print(f"Loss dtype: {loss.dtype}, Model dtype: {model.dtype}")
        # for param in model.parameters():
        #     if param.grad is not None:
        #         print(f"Grad dtype: {param.grad.dtype}, Model dtype: {model.dtype}")

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        train_bar.set_postfix(loss=loss.item())

    avg_train_loss = total_loss / len(train_dataloader)

    # 🚀 검증 실행
    avg_val_loss = validation_step(model, val_dataloader, avg_train_loss)

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")


Epoch 1/3:  87%|████████▋ | 36054/41472 [69:14:40<10:16:24,  6.83s/it, loss=0.977]

In [ ]:
import os

# 🚀 저장할 디렉토리 설정
save_directory = "fine_tuned_phi4_lora"
os.makedirs(save_directory, exist_ok=True)

# ✅ LoRA Adapter 저장
model.save_pretrained(save_directory)  # LoRA Adapter 저장
tokenizer.save_pretrained(save_directory)  # 토크나이저 저장

print(f"✅ Model and LoRA adapter saved at {save_directory}")


In [ ]:
import torch
import transformers
from sklearn.metrics import precision_score, recall_score, f1_score

# 🔹 Hugging Face Text Generation Pipeline 설정
pipeline = transformers.pipeline(
    "text-generation",
    model="unsloth/phi-4-unsloth-bnb-4bit",
    model_kwargs={"torch_dtype": "auto"},
    device_map="auto",
)

In [19]:


def evaluate_llm_generation(test_dataloader, all_genres, output_file="llm_predictions.txt"):
    """
    - `unsloth/phi-4-unsloth-bnb-4bit` 모델을 사용하여 장르를 예측하고 평가
    - `.txt` 파일에 `[Prompt] [LLM Predictions] [Answer]` 형식으로 저장
    - Precision, Recall, F1-score 계산 후 파일에 추가
    """
    all_preds = []
    all_labels = []

    with open(output_file, "w", encoding="utf-8") as f:
        f.write("LLM Movie Genre Prediction Results\n")
        f.write("=" * 100 + "\n\n")

    for batch in test_dataloader:
        batch = {k: v for k, v in batch.items()}  # 타이틀, 설명, 정답 포함

        # 🚀 프롬프트 메시지 생성 (대화형 메시지 포맷 적용)
        prompts = [
            [
                {"role": "system", "content": f"You are an AI movie genre classifier. Your task is to assign the most appropriate genres to a movie. Follow these rules: 1. Choose ONLY from the given genres: {', '.join(all_genres)}. 2. Output ONLY the predicted genres as a comma-separated list. 3. Do NOT repeat or copy the full genre list. 4. Do NOT add explanations or extra text."},
                {"role": "user", "content": f"Movie Title: {title}, Story: {desc}, Predicted Genres:"}
            ]
            for title, desc in zip(batch["title"], batch["description"])
        ]
        

        # 🚀 모델 예측 수행 (배치 단위로 처리)
        outputs = [pipeline(prompt, max_new_tokens=15)[0]["generated_text"][-1] for prompt in prompts]

        # 🚀 LLM이 생성한 장르 필터링 (올바른 장르만 포함)
        filtered_preds = []
        for pred in outputs:
            pred_text = pred['content']
            pred_lst = [genre.strip() for genre in pred_text.split(',')]
            # pred_genres = [genre for genre in all_genres if genre in pred_lst]  # 정해진 장르 목록에 포함된 것만 선택
            filtered_preds.append(pred_lst)

        # 🚀 실제 정답 가져오기
        actual_labels = batch["answer"]

        # 🚀 파일에 기록 (지정된 형식 적용)
        with open(output_file, "a", encoding="utf-8") as f:
            for prompt, pred, actual in zip(prompts, filtered_preds, actual_labels):
                f.write(f"[Prompt]\n{prompt}\n\n")
                f.write(f"[LLM Predictions]\n{', '.join(pred)}\n\n")
                f.write(f"[Answer]\n{', '.join(actual)}\n")
                f.write("-" * 100 + "\n\n")

        # 🚀 모델의 예측값을 리스트로 변환
        pred_vectors = [[1 if genre in pred else 0 for genre in all_genres] for pred in filtered_preds]
        label_vectors = [[1 if genre in actual else 0 for genre in all_genres] for actual in actual_labels]

        all_preds.extend(pred_vectors)
        all_labels.extend(label_vectors)

    # 🚀 Precision, Recall, F1-score 계산
    precision = precision_score(all_labels, all_preds, average="macro")
    recall = recall_score(all_labels, all_preds, average="macro")
    f1 = f1_score(all_labels, all_preds, average="macro")

    print(f"LLM Generation - Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

    # 🚀 평가 결과 파일에 저장
    with open(output_file, "a", encoding="utf-8") as f:
        f.write("\n" + "=" * 100 + "\n")
        f.write(f"Final Evaluation Metrics:\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1-score: {f1:.4f}\n")
        f.write("=" * 100 + "\n")

    return precision, recall, f1


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.50s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [20]:
evaluate_llm_generation(test_dataloader, all_genres)


LLM Generation - Precision: 0.4044, Recall: 0.6624, F1-score: 0.4574


(0.4044343079616163, 0.6624283520022937, 0.4574190720193066)